# Lecture 3: Spark Streaming

_Spark Streaming_ is an extension of the Spark API that enables scalabe stream processing.

The continous stream of input data can be ingested from many data sources such as **Kafka**, **Amazon s3** or **TCP sockets**. 

The Spark API allows to process data via high-level functions such as *map* and *reduce*. As we are going to see, it is also possible to use dataframe operations. 

Processed data can be exported to an external database and used to make live dashboards or offline analyses, or stored in files, or be used in a further stage of a Kafka pipeline. 

Spark streaming works by dividing the input data into _micro-batches_ that can be treated as static datasets. In Spark this is referred to as a *discretized stream* (*DStream*). The DStream is represented using RDDs.

![DStream](imgs/lecture3/DStream.png)

Any transformation applied on the DStream, i.e. anything like a `Dstream.map()`, will act independently on each batch. For example, in the image below, we can filter the original RDD to remove some data and produce a new stream. 

![DStream_filter](imgs/lecture3/Dstream_filter.png)

In this lecture we will see how to setup a simple stream using a TCP socket as a data source.

## Setup

In [1]:
# set this variable with one of the following values
# -> 'local'
# -> 'docker_container'
# -> 'docker_cluster'
CLUSTER_TYPE ='docker_cluster'

In [2]:
# set env variable
%env CLUSTER_TYPE $CLUSTER_TYPE

env: CLUSTER_TYPE=docker_cluster


## Start the cluster 

Environment variables need to be set only in the case of a local cluster

In [3]:
if CLUSTER_TYPE=='local':
    import findspark
    findspark.init('/home/pazzini/work/courses/MAPD_B/MAPD-B/spark/spark-3.2.1-bin-hadoop3.2')

In [4]:
%%script bash --no-raise-error

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    echo "Launching master and worker"
    
    # start master 
    $SPARK_HOME/sbin/start-master.sh --host localhost \
        --port 7077 --webui-port 8080
    
    # start worker
    $SPARK_HOME/sbin/start-worker.sh spark://localhost:7077 \
        --cores 4 --memory 2g
fi

## Create the Spark session

Later on we will explain what is the role of [Apache Arrow](), but first we need to install it and create the spark session with it.

In [5]:
from pyspark.sql import SparkSession

if CLUSTER_TYPE in ['local', 'docker_container']:
    
    spark = SparkSession.builder \
        .master("spark://localhost:7077")\
        .appName("Spark streaming application")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

elif CLUSTER_TYPE == 'docker_cluster':
    
    # use the provided master
    spark = SparkSession.builder \
        .master("spark://spark-master:7077")\
        .appName("Spark streaming application")\
        .config("spark.executor.memory", "512m")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/17 08:21:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
sc = spark.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=Spark streaming application>

## Spark _Streaming_ context

The first step of a Spark streaming application is the creation of a `StreamingContext`. 

It is a similar concept to the `sparkContext` but it requires to be initialized with some additional information on how to handle the micro-batches.

The context can be initialized using `StreamingContext(SparkContext, batch_interval`). 
The `batch_interval` parameter represents the wall-time between two batches, i.e. the batch duration in seconds.

There could be at most one `StreamingContext` for each Spark application. 

The processing start when `StreamingContext.start()` is called, and it can be stopped with `StreamingContext.stop()`. 

By default, if the `StreamingContext` is stoped without passing the `stopSparkContext=False` option, the sparkContext is also stopped (thus the application is closed).

In [7]:
from pyspark.streaming import StreamingContext

# create a streaming context with a batch interval of 2 seconds
ssc = StreamingContext(sc, 2) 

For this example spark will read data from a TCP socket. 

A dummy data stream is generated by a simple python program that can be found in `utils/producer.py`. 
When executed, the producer will write data on port `5555` of `localhost`. 

The dataset consists of fake credit card transactions.

Have a look at the code from the producer before executing it.

In [8]:
! cat utils/producer.py

import socket
import json
import time
import random
import argparse

first_names=('John','Andy','Joe','Alice')
last_names=('Johnson','Smith','Jones', 'Millers')

def send_messages(client_socket):
    try:
        while 1:
            msg = {
                'name': random.choice(first_names),
                'surname': random.choice(last_names),
                'amount': '{:.2f}'.format(random.random()*1000),
                'delta_t': '{:.2f}'.format(random.random()*10),
                'flag': random.choices([0,1], weights=[0.8, 0.2])[0]
            }
            client_socket.send((json.dumps(msg)+"\n").encode('utf-8'))
            time.sleep(0.1)

    except KeyboardInterrupt:
        exit()

if __name__ == "__main__":

    parser = argparse.ArgumentParser()
    parser.add_argument('--hostname', type=str, required=True)
    args = parser.parse_args()
    print('Using hostname:', args.hostname)

    new_skt = socket.socket()
    host = args.hostname
    port = 5555 
    new_skt.bind

The producer will generate new records in the form of a random combination of:
- firstname
- lastname
- amount
- delta time
- flag

These information will be formatted into a .json data format

To declare to Spark that the StreamingContext data source will be a TCP socket, located at a given `hostname` and `port`, we can use the following 

`socketTextStream(hostname, port)`

Have a look at the StreamingContext documentation to see all the available options, at the [link](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.streaming.StreamingContext.html)

In [9]:
# create a socket stream from the hostname where we collect data from
if CLUSTER_TYPE in ['local', 'docker_container']:
    hostname = "localhost"
    
elif CLUSTER_TYPE == 'docker_cluster':
    hostname = "spark-master"


# and set the port to 5555
socket_stream = ssc.socketTextStream(hostname, 5555)

#### start the python producer.py script

Depending on the Spark deployment mode for this exercise, do either of the following:

**Local** 
- from a terminal, run the python script with the option `--hostname localhost`


**Single Docker container**
- from a terminal, identify the id of the container using `docker ps`
- connect to the Docker container using the `docker exec -it <CONTAINER_ID> bash` command
- from the docker container, run the python script with the option `--hostname localhost`
  
**Docker cluster**
- from a terminal, identify the id of the `spark-master` container using `docker ps`
- connect to the Docker container using the `docker exec -it <CONTAINER_ID> bash` command
- from the docker container, run the python script with the option `--hostname spark-master`

The first thing we need to to is load the json describing each transaction.

In [10]:
import json

# use the map() transformation to apply the same function to all rdds
# the function we want to run is the json.load() of the messages
json_stream = socket_stream.map(lambda msg: json.loads(msg))

It is possible to print some elements of each batch with `pprint()`. This can be used to explore the RDDs.

In [11]:
json_stream.pprint()

Start the computations with `ssc.start()` and stop with `ssc.stop(stopSparkContext=False)`. Remember that after this last instruction the streaming context must be defined again.

In [12]:
ssc.start()

-------------------------------------------
Time: 2022-05-17 08:22:26
-------------------------------------------
{'name': 'John', 'surname': 'Jones', 'amount': '999.35', 'delta_t': '7.83', 'flag': 1}
{'name': 'John', 'surname': 'Millers', 'amount': '691.91', 'delta_t': '1.29', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '558.26', 'delta_t': '9.28', 'flag': 1}
{'name': 'Andy', 'surname': 'Jones', 'amount': '412.05', 'delta_t': '7.29', 'flag': 1}
{'name': 'John', 'surname': 'Jones', 'amount': '861.39', 'delta_t': '3.87', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '476.84', 'delta_t': '7.16', 'flag': 0}



-------------------------------------------
Time: 2022-05-17 08:22:28
-------------------------------------------
{'name': 'Andy', 'surname': 'Millers', 'amount': '493.26', 'delta_t': '2.52', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '883.92', 'delta_t': '6.42', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '716.36', 'delta_t': '8.71', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '966.70', 'delta_t': '6.81', 'flag': 1}
{'name': 'Joe', 'surname': 'Jones', 'amount': '509.31', 'delta_t': '7.73', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '174.35', 'delta_t': '7.12', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '524.66', 'delta_t': '0.95', 'flag': 0}
{'name': 'Alice', 'surname': 'Johnson', 'amount': '860.73', 'delta_t': '9.16', 'flag': 1}
{'name': 'John', 'surname': 'Smith', 'amount': '654.39', 'delta_t': '5.85', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '55.76', 'delta_t': '6.99', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 08:22:30
-------------------------------------------
{'name': 'John', 'surname': 'Smith', 'amount': '502.74', 'delta_t': '5.67', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '994.56', 'delta_t': '9.74', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '923.01', 'delta_t': '2.87', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '957.97', 'delta_t': '9.62', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '53.69', 'delta_t': '3.61', 'flag': 1}
{'name': 'Andy', 'surname': 'Smith', 'amount': '852.39', 'delta_t': '6.34', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '621.82', 'delta_t': '0.71', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '548.69', 'delta_t': '7.78', 'flag': 0}
{'name': 'Andy', 'surname': 'Smith', 'amount': '675.73', 'delta_t': '3.49', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '41.75', 'delta_t': '4.20', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 08:22:32
-------------------------------------------
{'name': 'John', 'surname': 'Millers', 'amount': '320.22', 'delta_t': '5.12', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '156.95', 'delta_t': '1.60', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '113.32', 'delta_t': '8.96', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '109.42', 'delta_t': '1.52', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '508.72', 'delta_t': '8.73', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '917.82', 'delta_t': '2.44', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '858.18', 'delta_t': '5.27', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '568.43', 'delta_t': '8.74', 'flag': 1}
{'name': 'John', 'surname': 'Johnson', 'amount': '916.86', 'delta_t': '8.22', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '666.68', 'delta_t': '8.64', 'flag': 

-------------------------------------------
Time: 2022-05-17 08:22:34
-------------------------------------------
{'name': 'Andy', 'surname': 'Smith', 'amount': '25.99', 'delta_t': '2.61', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '421.63', 'delta_t': '4.83', 'flag': 1}
{'name': 'Joe', 'surname': 'Jones', 'amount': '495.55', 'delta_t': '2.74', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '571.09', 'delta_t': '7.23', 'flag': 1}
{'name': 'Joe', 'surname': 'Smith', 'amount': '139.98', 'delta_t': '4.84', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '952.07', 'delta_t': '4.24', 'flag': 1}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '220.32', 'delta_t': '4.78', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '80.61', 'delta_t': '6.44', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '636.90', 'delta_t': '1.40', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '829.10', 'delta_t': '3.09', 'flag': 0}
...



...

-------------------------------------------
Time: 2022-05-17 08:24:42
-------------------------------------------
{'name': 'John', 'surname': 'Millers', 'amount': '334.72', 'delta_t': '7.14', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '88.13', 'delta_t': '4.80', 'flag': 1}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '9.39', 'delta_t': '5.43', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '204.09', 'delta_t': '1.69', 'flag': 1}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '312.94', 'delta_t': '3.40', 'flag': 1}
{'name': 'Joe', 'surname': 'Millers', 'amount': '960.20', 'delta_t': '9.62', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '241.05', 'delta_t': '1.58', 'flag': 1}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '606.50', 'delta_t': '9.15', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '527.44', 'delta_t': '5.68', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '684.42', 'delta_t': '0.33', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 08:24:44
-------------------------------------------
{'name': 'Alice', 'surname': 'Johnson', 'amount': '166.01', 'delta_t': '9.97', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '211.13', 'delta_t': '1.43', 'flag': 1}
{'name': 'Alice', 'surname': 'Millers', 'amount': '864.34', 'delta_t': '0.95', 'flag': 1}
{'name': 'Alice', 'surname': 'Smith', 'amount': '241.74', 'delta_t': '4.84', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '784.31', 'delta_t': '3.25', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '39.83', 'delta_t': '2.77', 'flag': 1}
{'name': 'Alice', 'surname': 'Jones', 'amount': '404.90', 'delta_t': '9.65', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '416.96', 'delta_t': '8.90', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '19.11', 'delta_t': '3.06', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '309.08', 'delta_t': '0.38', 'flag': 0}
...

-------------------------------------------
Time: 2022-05-17 08:24:46
-------------------------------------------
{'name': 'Joe', 'surname': 'Smith', 'amount': '792.63', 'delta_t': '4.43', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '79.74', 'delta_t': '0.19', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '289.81', 'delta_t': '7.78', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '481.41', 'delta_t': '7.58', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '372.48', 'delta_t': '2.25', 'flag': 0}
{'name': 'Andy', 'surname': 'Johnson', 'amount': '422.58', 'delta_t': '1.18', 'flag': 0}
{'name': 'Joe', 'surname': 'Smith', 'amount': '587.43', 'delta_t': '8.99', 'flag': 1}
{'name': 'Joe', 'surname': 'Jones', 'amount': '89.28', 'delta_t': '9.20', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '742.30', 'delta_t': '5.51', 'flag': 0}
{'name': 'Alice', 'surname': 'Smith', 'amount': '309.65', 'delta_t': '3.08', 'flag': 1}
...



-------------------------------------------
Time: 2022-05-17 08:24:48
-------------------------------------------
{'name': 'Alice', 'surname': 'Smith', 'amount': '58.10', 'delta_t': '5.66', 'flag': 0}
{'name': 'John', 'surname': 'Millers', 'amount': '283.50', 'delta_t': '5.13', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '887.81', 'delta_t': '4.98', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '431.40', 'delta_t': '3.28', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '840.34', 'delta_t': '0.06', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '132.42', 'delta_t': '5.11', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '724.60', 'delta_t': '8.34', 'flag': 0}
{'name': 'John', 'surname': 'Smith', 'amount': '927.52', 'delta_t': '6.73', 'flag': 0}
{'name': 'Andy', 'surname': 'Jones', 'amount': '968.11', 'delta_t': '7.86', 'flag': 0}
{'name': 'Alice', 'surname': 'Millers', 'amount': '216.47', 'delta_t': '1.04', 'flag': 0}
...



-------------------------------------------
Time: 2022-05-17 08:24:50
-------------------------------------------
{'name': 'John', 'surname': 'Millers', 'amount': '159.53', 'delta_t': '2.91', 'flag': 0}
{'name': 'Andy', 'surname': 'Millers', 'amount': '321.52', 'delta_t': '1.22', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '957.11', 'delta_t': '1.83', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '550.13', 'delta_t': '9.68', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '290.61', 'delta_t': '7.00', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '816.65', 'delta_t': '4.60', 'flag': 1}
{'name': 'Alice', 'surname': 'Jones', 'amount': '494.90', 'delta_t': '2.32', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '655.73', 'delta_t': '8.60', 'flag': 0}
{'name': 'John', 'surname': 'Johnson', 'amount': '151.88', 'delta_t': '9.45', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '113.81', 'delta_t': '7.93', 'flag': 0}
...

In [13]:
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2022-05-17 08:24:52
-------------------------------------------
{'name': 'Andy', 'surname': 'Johnson', 'amount': '701.87', 'delta_t': '9.73', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '681.78', 'delta_t': '9.46', 'flag': 0}
{'name': 'John', 'surname': 'Jones', 'amount': '115.43', 'delta_t': '2.18', 'flag': 0}
{'name': 'Joe', 'surname': 'Millers', 'amount': '429.87', 'delta_t': '8.95', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '862.95', 'delta_t': '9.96', 'flag': 1}
{'name': 'Andy', 'surname': 'Millers', 'amount': '617.61', 'delta_t': '8.99', 'flag': 1}
{'name': 'John', 'surname': 'Johnson', 'amount': '877.68', 'delta_t': '5.12', 'flag': 0}
{'name': 'Joe', 'surname': 'Jones', 'amount': '291.51', 'delta_t': '1.55', 'flag': 0}
{'name': 'Joe', 'surname': 'Johnson', 'amount': '101.30', 'delta_t': '5.57', 'flag': 0}
{'name': 'Alice', 'surname': 'Jones', 'amount': '231.11', 'delta_t': '2.59', 'flag': 0}
...



22/05/17 08:24:52 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
22/05/17 08:24:52 ERROR TaskSchedulerImpl: Lost executor 3 on 172.19.0.7: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


Now that we know we can stream data to Spark we would like to start performing basic operations in a distributed fashion.

Once the SparkStreamingContext is stopped we have to re-create a new one, as the connection between the socket and Spark is lost.

Before running the following cells we should therefore:
1. create a new `scc` object
2. point it to the proper TCP socket and port
3. start again the python producer application

In [14]:
# create a new Spark StreamingContext with a batch wall-time of 2 seconds
ssc = StreamingContext(sc, 2) 

In [15]:
# define the socket stream using the appropriate endpoint and port
socket_stream = ssc.socketTextStream(hostname, 5555)

#### start once again the python producer script

We now start listening on the TCP socket, interpreting the input data stream as json loads.

Remember to get rid of the `pprint()` action, that would otherwise be performed continously, dumping the input data into the Jupyter cells.

In [16]:
# create a new json_stream object by reading the json loads from the socket
json_stream = socket_stream.map(lambda msg: json.loads(msg))

We may want to convert each batch into a Spark DataFrame to have access to the higher level APIs. 

In order to do that, let's first convert all the numeric features of the json into python floats and integers variables. 
This is a simple type cast operation in python, that can be easily parallelized.

The python dictionary produced by each json message, re-casted with the proper data types, can then be used to create a `Row`for each transaction.

In [17]:
from pyspark.sql import Row

# create a row for each message 
#   convert each numerical value to the proper python type
#   create a row from each message
def create_row_rdd(t):
    t['amount'] = float(t['amount'])
    t['delta_t'] = float(t['delta_t'])
    t['flag'] = int(t['flag'])
    
    return Row(**t)

# apply the transformation to the json_stream rdd
row_stream = json_stream.map(create_row_rdd)

The method `DStream.foreachRDD` can be used to apply custom transformations to each *batch* of data. 

In our case, we are insterested in converting each batch of data into a Spark dataframe and perform operations, such as finding the number of transactions for each user. 

For the scope of this simple use-case we can consider that all batches where a user has performed more than one transaction with the `flag` field equal to one can be identified as fraudulent.

In [18]:
# if left unconstrained, Spark might want to create a very 
# large number of partitions for this streaming application
# 
# using way more partitions than necessary always results in
# a huge over-head due to the partition-to-partition communications
# 
# this line is a trick to force Spark to use a small number of partitions
# thus making it more efficient in the case of small workloads and few executors

spark.conf.set("spark.sql.shuffle.partitions", 4)

In [19]:
from pyspark.sql.functions import concat, col, lit, countDistinct

# process each bach to identify possibly fraudolent transactions
#   1. convert the rdd into a dataframe (provide the schema if necessary)
#   2. compute the number of transactions per batch per user (a unique combination of first_and_lastname)
#   3. identify all the "suspicios" transactions by counting the number of transactions from a unique user 
#   3. print the resulting dataframe to mimick a live reporting of the suspicious transactions

def process_batch(rdd):
    # convert rdd to df
    #   check the documentation and/or the Lecture2 notebook 
    #   for details on how to create and pass a schema to a dataframe   
    df = rdd.toDF(
        schema='name string, surname string, amount float, delta_t float, flag int'
    )
    
    # find number of transactions for each user when flag = 1 
    #    declare a new column to create a unique user identifier 
    #    this can be easily done by concatenating first- and last-name fields
    #    check the concat function from pyspark.sql.functions 
    num_transactions = (
        df
        .where(col('flag')==1)
        .withColumn('id', concat(col('name'), col('surname')))
        .groupBy('id')
        .count()
    )
    
    # find suspicious transactions
    #    filter only users with more than one transaction per batch
    #    create a "fraud" column with a value of 1 for the selected users (check the lit function)
    #    from the dataframe, project only the unique id and fraud columns
    sus_transactions = (
        num_transactions
        .where(col('count')>1)
        .withColumn('fraud', lit(1))
        .select(col('id'), col('fraud'))
    )
    
    # (trigger an automatic alert)
    # print the first 5 items of the resulting dataframe
    sus_transactions.show(5)

Finally, instruct Spark to execute this `process_batch` function for each RDD you will have in your DStream

In [20]:
row_stream.foreachRDD(process_batch)

Now you should be ready to start the spark streaming context

In [21]:
ssc.start()

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+---+-----+
| id|fraud|
+---+-----+
+---+-----+

+------------+-----+
|          id|fraud|
+------------+-----+
|AliceMillers|    1|
+------------+-----+



+------------+-----+
|          id|fraud|
+------------+-----+
|AliceJohnson|    1|
+------------+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



...

+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+---------+-----+
|       id|fraud|
+---------+-----+
|JohnSmith|    1|
+---------+-----+



+---+-----+
| id|fraud|
+---+-----+
+---+-----+



+----------+-----+
|        id|fraud|
+----------+-----+
|JoeMillers|    1|
|  JoeSmith|    1|
+----------+-----+



+------------+-----+
|          id|fraud|
+------------+-----+
|AliceJohnson|    1|
+------------+-----+



In [ ]:
# stop streaming context
ssc.stop(stopSparkContext=False)

## Stop worker and master

In [ ]:
sc.stop()
spark.stop()

In [ ]:
%%script bash --no-raise-error

if [[ "$CLUSTER_TYPE" != "docker_cluster" ]]; then
    # stop worker 
    $SPARK_HOME/sbin/stop-worker.sh
    
    # start master
    $SPARK_HOME/sbin/stop-master.sh
fi